<a href="https://colab.research.google.com/github/vantuyenvhp/Timekeeping/blob/main/ChamCong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyzk
# Kết nối đến máy chấm công và lấy thông tin.
from zk import ZK, const
from datetime import datetime, timedelta

def thong_tin_may_cham_cong(ip: str, port: int):
    zk = ZK(ip, port=port, timeout=5, password=0, force_udp=False, ommit_ping=True)
    try:
        print(f"\nĐang kết nối đến máy chấm công tại {ip}:{port}...")
        conn = zk.connect()
        print("Kết nối thành công!\n")

        while True:
            print("Bạn muốn thực hiện thao tác nào?\n")
            print("1. Lấy thông tin máy chấm công")
            print("===============================================")
            print("2. Lấy danh sách người dùng")
            print("===============================================")
            print("3. Lấy dữ liệu chấm công của tất cả người dùng")
            print("===============================================")
            print("4. Lọc dữ liệu chấm công theo User ID")
            print("===============================================")
            print("5. Cập nhật thông tin user")
            print("===============================================")
            print("6. Xóa người dùng")
            print("===============================================")
            print("7. Cập nhật thời gian")
            print("===============================================")
            print("0. Thoát")
            print("===============================================")
            lua_chon = input("Nhập lựa chọn (0-7): ")

            if lua_chon == "1":
                # Lấy thông tin máy chấm công
                info = conn.get_device_name()
                firmware = conn.get_firmware_version()
                platform = conn.get_platform()
                serial = conn.get_serialnumber()

                print("\n=== Thông tin máy chấm công ===")
                print(f"Tên thiết bị     : {info}")
                print(f"Firmware version : {firmware}")
                print(f"Nền tảng         : {platform}")
                print(f"Số seri          : {serial}\n")

            elif lua_chon == "2":
                # Lấy danh sách người dùng
                users = conn.get_users()
                print("\n=== Danh sách người dùng ===")
                for user in users:
                    print(f"UserID: {user.user_id}, Tên: {user.name}, Card: {user.card}, Group ID: {user.group_id}")
                print()

            elif lua_chon == "3":
                # Lấy dữ liệu chấm công
                print("\nChọn khoảng thời gian muốn lấy dữ liệu chấm công:")
                print("1. 1 tháng")
                print("2. 6 tháng")
                print("3. 1 năm")
                print("4. Toàn bộ")
                print("5. Hôm nay")
                thoi_gian = input("Nhập lựa chọn (1-5): ")

                now = datetime.now()
                if thoi_gian == "1":
                    start_time = now - timedelta(days=30)
                elif thoi_gian == "2":
                    start_time = now - timedelta(days=180)
                elif thoi_gian == "3":
                    start_time = now - timedelta(days=365)
                elif thoi_gian == "4":
                    start_time = None  # Toàn bộ
                else:
                    start_time = now - timedelta(days=1)

                users = conn.get_users()
                user_dict = {user.user_id: user.name for user in users}
                attendance = conn.get_attendance()
                print("\n=== Dữ liệu chấm công ===")
                for record in attendance:
                    if start_time is None or record.timestamp >= start_time:
                        user_name = user_dict.get(record.user_id, "Không rõ tên")
                        print(f"UserID: {record.user_id}, Tên: {user_name}, Thời gian: {record.timestamp}")
                print()

            elif lua_chon == "4":
                # Lọc dữ liệu chấm công theo User ID
                user_id = input("Nhập User ID cần lọc: ").strip()

                print("\nChọn khoảng thời gian:")
                print("1. 1 ngày")
                print("2. 1 tuần")
                print("3. 1 tháng")
                print("4. 6 tháng")
                thoi_gian = input("Nhập lựa chọn (1-4): ")

                now = datetime.now()
                if thoi_gian == "1":
                    start_time = now - timedelta(days=1)
                elif thoi_gian == "2":
                    start_time = now - timedelta(weeks=1)
                elif thoi_gian == "3":
                    start_time = now - timedelta(days=30)
                elif thoi_gian == "4":
                    start_time = now - timedelta(days=180)
                else:
                    print("Lựa chọn không hợp lệ.\n")
                    continue

                users = conn.get_users()
                user_dict = {user.user_id: user.name for user in users}
                attendance = conn.get_attendance()

                print(f"\n=== Dữ liệu chấm công của UserID {user_id} ===")
                found = False
                for record in attendance:
                    if str(record.user_id) == user_id and record.timestamp >= start_time:
                        user_name = user_dict.get(record.user_id, "Không rõ tên")
                        print(f"Thời gian: {record.timestamp}, Tên: {user_name}")
                        found = True
                if not found:
                    print("Không có bản ghi nào phù hợp.")
                print()

            elif lua_chon == "5":
                # Cập nhật thông tin user
                user_id = input("Nhập User ID cần cập nhật: ").strip()
                users = conn.get_users()
                user_dict = {user.user_id: user.name for user in users}

                if user_id in user_dict:
                    print(f"User hiện tại: UserID: {user_id}, Tên: {user_dict[user_id]}")
                    new_name = input("Nhập tên mới để cập nhật: ").strip()
                    if new_name:
                        conn.set_user(user_id=user_id, name=new_name)
                        print("Cập nhật thông tin thành công!")
                    else:
                        print("Tên mới không được để trống. Hủy thao tác cập nhật.")
                else:
                    print(f"Không tìm thấy UserID: {user_id} trong danh sách người dùng.")

            elif lua_chon == "6":
                # Xóa người dùng
                user_id = input("Nhập User ID cần xóa: ").strip()
                users = conn.get_users()
                user_dict = {user.user_id: user.name for user in users}

                if user_id in user_dict:
                    user_name = user_dict[user_id]
                    print(f"Bạn có chắc chắn muốn xóa UserID: {user_id}, Tên: {user_name}?")
                    print("1. Xóa")
                    print("0. Hủy")
                    confirm = input("Nhập lựa chọn (1 để xóa, 0 để hủy): ")
                    if confirm == "1":
                        conn.delete_user(user_id=user_id)
                        print(f"Đã xóa thành công UserName: {user_name}.\n")
                    else:
                        print("Hủy thao tác xóa.")
                else:
                    print(f"Không tìm thấy UserID: {user_id} trong danh sách người dùng.")

            elif lua_chon == "7":
                # Cập nhật thời gian
                print("\nBạn muốn cập nhật thời gian nào?\n")
                print("1. Thời gian hiện tại")
                print("2. Thời gian tùy chọn")
                thoi_gian_chon = input("Nhập lựa chọn (1-2): ").strip()
                if thoi_gian_chon == "1":
                    # Cập nhật thời gian hiện tại
                    current_time = datetime.now() # - timedelta(minutes=10)
                    conn.set_time(current_time)
                    print(f"Đã cập nhật thời gian hiện tại thành công! Thời gian mới:  \n{current_time}\n")
                elif thoi_gian_chon == "2":
                    # Cập nhật thời gian tùy chọn
                    try:
                        custom_time_str = input("Nhập thời gian (định dạng: YYYY-MM-DD HH:MM:SS): ").strip()
                        custom_time = datetime.strptime(custom_time_str, "%Y-%m-%d %H:%M:%S")
                        conn.set_time(custom_time)
                        print("Đã cập nhật thời gian tùy chọn thành công!")
                    except ValueError:
                        print("Thời gian nhập không hợp lệ. Vui lòng thử lại.")
                else:
                    print("Lựa chọn không hợp lệ. Hủy thao tác cập nhật thời gian.")

            elif lua_chon == "0":
                print("Đã thoát chương trình.")
                break
            else:
                print("Lựa chọn không hợp lệ. Vui lòng chọn lại.\n")

        conn.disconnect()
    except Exception as e:
        print(f"Lỗi khi kết nối: {e}")

# === Bắt đầu chương trình chính ===
if __name__ == "__main__":
    print("===== CHƯƠNG TRÌNH KẾT NỐI MÁY CHẤM CÔNG =====")
    ip = input("Nhập địa chỉ IP của máy chấm công(Ví dụ: 192.168.1.18): ")
    port = 4370  # Port mặc định cho thiết bị ZKTeco
    thong_tin_may_cham_cong(ip, port)

# VPHCM 	115.79.195.199
# VPHCM	    115.79.199.64
# Long An	113.161.187.2
# Long An	115.75.184.19
# TTDVKT	115.73.220.144
# TTDVKT	14.161.29.177

  Preparing metadata (setup.py) ... done
  Created wheel for pyzk: filename=pyzk-0.9-py3-none-any.whl size=16925 sha256=8e9bf28552ae6e3bd863ae5264d73bc520afebab166b27f112ed848516c8252c
  Stored in directory: /root/.cache/pip/wheels/cb/a5/dc/a024f6dbd2d30c3c34910632ea41d9deed6a072d6b9d79b4db
Successfully built pyzk
===== CHƯƠNG TRÌNH KẾT NỐI MÁY CHẤM CÔNG =====
Nhập địa chỉ IP của máy chấm công(Ví dụ: 192.168.1.18): 10.2.2.62

Đang kết nối đến máy chấm công tại 10.2.2.62:4370...
Lỗi khi kết nối: timed out
